# [AlexNet]

*KU LeeDongGyu*

### Contents

1. Almost Original Alexnet
2. My Alexnet
```
1) Size = 64,
2) Size = 48,
```
3. Size = 48, No Early Stopping
```
1) Epoch = 50
2) Epoch = 100
```

### Install Packages

In [0]:
pip install opencv-python

### Module

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [0]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [0]:
from lrn import LRN #만든 모듈, class
from f1score import macro_f1score, weighted_f1score
from pool_helper import PoolHelper

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

## 1. Almost Original Alexnet

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)


y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 227
x_train = np.array(x_train).reshape([-1,48,48,1])

In [0]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_train_zoom.shape

(28698, 227, 227, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_valid_zoom.shape

(3589, 227, 227, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_test_zoom.shape

(3588, 227, 227, 1)

In [0]:
# Alexnet을 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.
# 출처 : https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py

def Alexnet(img_shape=(227, 227, 1), n_classes=10, l2_reg=0.,weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape, strides=4,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer1_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer2_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))
    
    # Layer 3
    alexnet.add(Conv2D(384, (3, 3), padding='same', strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer3_LRN'))
    
    # Layer 4
    alexnet.add(Conv2D(384, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer4_LRN'))

    # Layer 5
    alexnet.add(Conv2D(256, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer5_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(Activation('softmax'))

    if weights is not None:
        alexnet.load_weights(weights)

    return alexnet

In [0]:
model = Alexnet(img_shape=(227, 227, 1), n_classes=7, l2_reg=0.,weights=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#모수가 데이터에 비해 굉장히 많지만, 일단 진행
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        11712     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 27, 27, 256)       0

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=10,callbacks=[early_stopping])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 28698 samples, validate on 3589 samples
Epoch 1/10
28698/28698 [==============================] - 3093s 108ms/sample - loss: 1.8708 - acc: 0.2479 - macro_f1score: 1.7897e-04 - weighted_f1score: 2.9362e-05 - val_loss: 1.8170 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/10
28698/28698 [==============================] - 3085s 108ms/sample - loss: 1.8144 - acc: 0.2513 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8151 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/10
28698/28698 [==============================] - 3086s 108ms/sample - loss: 1.8125 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8178 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/10
28698/28698 [==========================

Too many parameters compared to the data. <br>
So it doesn't work well.

## 2. My Alexnet

In [0]:
# Alexnet을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
size = 64 #적당한 크기로 잡음.
x_train = np.array(x_train).reshape([-1,48,48,1])

In [0]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")
for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_train_zoom.shape

(28698, 64, 64, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_valid_zoom.shape

(3589, 64, 64, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_test_zoom.shape

(3588, 64, 64, 1)

#### 1) Size = 64

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 14 x 14 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(64, 64, 1), n_classes=7, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(64, 64, 1), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 24)        624       
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 30, 30, 24)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        13888     
_________________________________________________________________
activation_9 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 14, 14, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 196s 7ms/sample - loss: 1.8248 - acc: 0.2443 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8159 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 196s 7ms/sample - loss: 1.8148 - acc: 0.2509 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8133 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 194s 7ms/sample - loss: 1.7927 - acc: 0.2532 - macro_f1score: 0.0019 - weighted_f1score: 2.2977e-04 - val_loss: 1.7458 - val_acc: 0.2767 - val_macro_f1score: 0.0062 - val_weighted_f1score: 6.2836e-04
Epoch 4/100
28698/28698 [==============================] - 194s 7ms/sample - loss: 1.6952 - acc: 0.3110 - macro_f1score: 0.0543 - weighted_f1score: 0.0075 - v

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 8s 2ms/sample - loss: 1.2813 - acc: 0.5538 - macro_f1score: 0.4167 - weighted_f1score: 0.0694

Accuracy: 0.5538, Macro F1 Score: 0.4167, Weighted F1 Score: 0.0694


#### 2) Size = 48

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 10 x 10 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(48, 48, 1), n_classes=7, l2_reg=0., weights=None):

	# Initialize model
	alexnet = Sequential()


	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 1), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 22, 22, 24)        624       
_________________________________________________________________
activation_32 (Activation)   (None, 22, 22, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 22, 22, 24)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 24)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 10, 10, 64)        13888     
_________________________________________________________________
activation_33 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 10, 10, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 97s 3ms/sample - loss: 1.9048 - acc: 0.2390 - macro_f1score: 0.0022 - weighted_f1score: 3.8506e-04 - val_loss: 1.8137 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.7417 - acc: 0.2885 - macro_f1score: 0.0408 - weighted_f1score: 0.0103 - val_loss: 1.6544 - val_acc: 0.3369 - val_macro_f1score: 0.0780 - val_weighted_f1score: 0.0196
Epoch 3/100
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.6641 - acc: 0.3336 - macro_f1score: 0.0756 - weighted_f1score: 0.0187 - val_loss: 1.6205 - val_acc: 0.3522 - val_macro_f1score: 0.0873 - val_weighted_f1score: 0.0223
Epoch 4/100
28698/28698 [==============================] - 96s 3ms/sample - loss: 1.6079 - acc: 0.3632 - macro_f1score: 0.1028 - weighted_f1score: 0.0228 - val_loss: 1.5585 - val_acc: 0.390

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 3s 855us/sample - loss: 1.4436 - acc: 0.5061 - macro_f1score: 0.3665 - weighted_f1score: 0.0618

Accuracy: 0.5061, Macro F1 Score: 0.3665, Weighted F1 Score: 0.0618


## 3. Size = 48 , No Early Stopping
---

### 1) Epoch = 50

In [0]:
### early stopping 없이 ###

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 1), n_classes=7, l2_reg=0.,weights=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.8570 - acc: 0.2407 - macro_f1score: 0.0026 - weighted_f1score: 4.5010e-04 - val_loss: 1.7650 - val_acc: 0.2686 - val_macro_f1score: 6.1576e-04 - val_weighted_f1score: 7.2160e-05
Epoch 2/50
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.6773 - acc: 0.3249 - macro_f1score: 0.0771 - weighted_f1score: 0.0131 - val_loss: 1.6173 - val_acc: 0.3511 - val_macro_f1score: 0.1425 - val_weighted_f1score: 0.0261
Epoch 3/50
28698/28698 [==============================] - 94s 3ms/sample - loss: 1.5590 - acc: 0.3956 - macro_f1score: 0.1386 - weighted_f1score: 0.0261 - val_loss: 1.5232 - val_acc: 0.4152 - val_macro_f1score: 0.1499 - val_weighted_f1score: 0.0306
Epoch 4/50
28698/28698 [==============================] - 94s 3ms/sample - loss: 1.4749 - acc: 0.4

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 3s 824us/sample - loss: 2.5452 - acc: 0.5059 - macro_f1score: 0.4456 - weighted_f1score: 0.0704

Accuracy: 0.5059, Macro F1 Score: 0.4456, Weighted F1 Score: 0.0704


### 2) Epoch = 100

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 1), n_classes=7, l2_reg=0.,weights=None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.8849 - acc: 0.2391 - macro_f1score: 0.0022 - weighted_f1score: 4.3057e-04 - val_loss: 1.7575 - val_acc: 0.2705 - val_macro_f1score: 0.0026 - val_weighted_f1score: 6.6626e-04
Epoch 2/100
28698/28698 [==============================] - 94s 3ms/sample - loss: 1.6683 - acc: 0.3367 - macro_f1score: 0.0790 - weighted_f1score: 0.0169 - val_loss: 1.5519 - val_acc: 0.3892 - val_macro_f1score: 0.1321 - val_weighted_f1score: 0.0266
Epoch 3/100
28698/28698 [==============================] - 94s 3ms/sample - loss: 1.5349 - acc: 0.4022 - macro_f1score: 0.1489 - weighted_f1score: 0.0286 - val_loss: 1.4701 - val_acc: 0.4335 - val_macro_f1score: 0.1473 - val_weighted_f1score: 0.0285
Epoch 4/100
28698/28698 [==============================] - 95s 3ms/sample - loss: 1.4680 - acc: 0.4290 - macro_f1score: 0.1783 - weighted_f1score: 0.0335 - val_loss: 1.4311 - val_acc: 0.4547 - 

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 3s 889us/sample - loss: 2.8512 - acc: 0.5298 - macro_f1score: 0.4614 - weighted_f1score: 0.0738

Accuracy: 0.5298, Macro F1 Score: 0.4614, Weighted F1 Score: 0.0738
